### Set-up

In [2]:
%pip uninstall logikon -y

Found existing installation: logikon 0.1.0.dev0
Uninstalling logikon-0.1.0.dev0:
  Successfully uninstalled logikon-0.1.0.dev0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

GH_ACCESS_TOKEN = getpass.getpass("GH_ACCESS_TOKEN: ")
os.environ["GH_ACCESS_TOKEN"] = GH_ACCESS_TOKEN

In [4]:
# install logikon module from github repo
%pip install hatch
%pip install git+https://$GH_ACCESS_TOKEN@github.com/logikon-ai/logikon.git

Note: you may need to restart the kernel to use updated packages.
  Cloning https://****@github.com/logikon-ai/logikon.git to /tmp/pip-req-build-nkivy5fd
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/logikon-ai/logikon.git' /tmp/pip-req-build-nkivy5fd
  Resolved https://****@github.com/logikon-ai/logikon.git to commit dcb97fa75358edcb0002c32ae365eb240232c145
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for logikon: filename=logikon-0.1.0.dev0-py3-none-any.whl size=22527 sha256=0f408d568b0e44b85b71774cc6a3f5412d4012b96ba1848d3d524b8a978d4326
  Stored in directory: /tmp/pip-ephem-wheel-cache-drfniyh7/wheels/10/bc/0f/621fa7ad65eb758157ed6c6ae76ca3429cca25fbbebd263f3f
Successfully built logikon
Note: you may need to restart the kernel to use updated packages.


### Imports and Api Keys

In [5]:
import getpass
import os

import logikon

OPENAI_API_KEY = getpass.getpass("OPENAI_API_KEY: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

HF_ACCESS_TOKEN = getpass.getpass("HF_ACCESS_TOKEN: ")
os.environ["HF_ACCESS_TOKEN"] = HF_ACCESS_TOKEN

### Data

In [6]:
# slightly adapted from `logikon/oasst1-delib` dataset

prompt = """prompter:
I live in the US, and i am thinking of moving to Argentina.
What pro and cons you think i will have?

assistant:
"""
completion = """Here are some pros and cons to consider if you are thinking of moving to Argentina:
Pros: 
Rich culture: Argentina has a rich cultural heritage, including music, dance, literature, and art.
Beautiful natural scenery: The country has a varied geography, from the Andes mountains to the fertile plains of the Pampas, and plenty of natural parks and reserves.
Good food: Argentine cuisine is known for its grilled meats and various types of pasta, and there are many great restaurants to be found throughout the country.
Low cost of living: In comparison to the United States, the cost of living in Argentina can be quite low, especially in terms of housing and food.
Cons:
Economic instability: Argentina has a history of economic instability, with periodic devaluations of its currency, high inflation, and government default on its debt.
High crime rates: Some areas of Argentina have high crime rates, particularly in the larger cities, so it's important to be cautious and aware of your surroundings.
Language barrier: Although many people in Argentina speak some English, it's still a good idea to learn Spanish if you plan to live there, as it may be difficult to get by without it in some areas.
Distance from home: Moving to a foreign country can be a major adjustment, especially if you are far from friends and family.
These are some of the factors to consider if you're thinking of moving to Argentina.
Ultimately, the decision to move will depend on your individual circumstances and priorities."""

### Example Usage 1:<br/>Create Informal Argument Map with OpenAI (Default Configuration)

In [7]:
config = logikon.DebugConfig(expert_model='gpt-3.5-turbo-instruct')
print(config)

expert_model='gpt-3.5-turbo-instruct' expert_model_kwargs={'temperature': 0.9} llm_framework='OpenAI' generation_kwargs=None metrics=[] artifacts=['informal_argmap'] report_to=[]


In [8]:
results = logikon.score(prompt, completion, config=config)



> Entering new ClaimExtractionChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment
Identify the key question addressed in a text.

# Inputs
Use the following inputs (a TEXT) to solve your assignment.

TEXT:
:::
prompter:
I live in the US, and i am thinking of moving to Argentina.
What pro and cons you think i will have?

assistant:

Here are some pros and cons to consider if you are thinking of moving to Argentina:
Pros: 
Rich culture: Argentina has a rich cultural heritage, including music, dance, literature, and art.
Beautiful natural scenery: The country has a varied geography, from the Andes mountains to the fertile plains of the Pampas, and plenty of natural parks and reserves.
Good food: Argentine cuisine is known for its grilled meats and various types of pasta, and there are m

In [9]:
results

DebugResults(artifacts=[Artifact(id='claims', description='Key claims in the deliberation', data=['Pros: Rich culture, beautiful natural scenery, good food, and low cost of living. Cons: Economic instability, high crime rates, language barrier, and distance from home.', 'Moving to Argentina may provide opportunities for new experiences and personal growth.', "The country's political instability can be a major con for those looking for a stable and secure environment.", 'The cost of moving to Argentina, including visa expenses and international travel, may also be a factor to consider.'], dtype=None, metadata=None), Artifact(id='informal_argmap', description='Informal Argument Map', data={'nodelist': [{'id': '2e8b6df8-5609-4e0b-81c1-0ac27bbacbaf', 'text': 'Pros: Rich culture, beautiful natural scenery, good food, and low cost of living. Cons: Economic instability, high crime rates, language barrier, and distance from home.', 'label': 'Claim-1', 'annotations': [], 'nodeType': 'propositio

In [11]:
results.artifacts[1].data

{'nodelist': [{'id': '2e8b6df8-5609-4e0b-81c1-0ac27bbacbaf',
   'text': 'Pros: Rich culture, beautiful natural scenery, good food, and low cost of living. Cons: Economic instability, high crime rates, language barrier, and distance from home.',
   'label': 'Claim-1',
   'annotations': [],
   'nodeType': 'proposition'},
  {'id': '4662759a-fa2c-4d55-9f67-501eb02bf14b',
   'text': 'Moving to Argentina may provide opportunities for new experiences and personal growth.',
   'label': 'Claim-2',
   'annotations': [],
   'nodeType': 'proposition'},
  {'id': 'f6afccce-940c-4d82-9265-ee9869e4fbe8',
   'text': "The country's political instability can be a major con for those looking for a stable and secure environment.",
   'label': 'Claim-3',
   'annotations': [],
   'nodeType': 'proposition'},
  {'id': 'a752d07a-7cc6-4cff-afa8-4548d391eeb8',
   'text': 'The cost of moving to Argentina, including visa expenses and international travel, may also be a factor to consider.',
   'label': 'Claim-4',
 

In [14]:
len(results.artifacts[1].data["nodelist"])

37